### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression
#from tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPS import CMAPSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
#from data_handler_MNIST import MNISTDataHandler
#from data_handler_CMAPS import CMAPSDataHandler

Using TensorFlow backend.


### Load cmaps data handler

In [2]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [3]:
def save_best_models(best_models_list, global_best_model_index, saveto, train_epochs=100):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = ind_model.tModel
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best
    tModel = best_models_list[global_best_model_index].tModel
    print(tModel.data_handler.data_scaler)
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    tModel.evaluate_model(['rhs', 'rmse'], round=2)
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Test on MNIST

In [4]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
#dhandler_cmaps, input_shape = cmaps_dhandler()

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

save_best_models(global_best_list, global_best_index, 'best_models/mnist/', train_epochs=200)

Launching experiment 1

Generation 1
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 1016)              797560    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1016)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10170     
Total params: 807,730
Trainable params: 807,730
Non-trainable params: 0
_________________________________________________________________


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 1s 20us/step - loss: 0.6970 - acc: 0.7909 - val_loss: 0.3830 - val_acc: 0.8868
Epoch 2/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.3649 - acc: 0.8931 - val_loss: 0.3317 - val_acc: 0.9021
Epoch 3/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.3114 - acc: 0.9091 - val_loss: 0.2881 - val_acc: 0.9160
Epoch 4/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.2710 - acc: 0.9210 - val_loss: 0.2487 - val_acc: 0.9274
Epoch 5/5
12000/12000 [==============================] - 0s 14us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 352)               276320    
_________________________________________________________________
dense_2 (Dense)              (None, 400)               141

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 10us/step - loss: 1.0727 - acc: 0.6953 - val_loss: 0.4464 - val_acc: 0.8808
Epoch 2/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.3705 - acc: 0.8952 - val_loss: 0.2990 - val_acc: 0.9129
Epoch 3/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.2814 - acc: 0.9171 - val_loss: 0.2574 - val_acc: 0.9207
Epoch 4/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.2291 - acc: 0.9323 - val_loss: 0.2237 - val_acc: 0.9346
Epoch 5/5
12000/12000 [==============================] - 0s 15us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 208)               163280    
_________________________________________________________________
dense_5 (Dense)              (None, 848)               177

...

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 94us/step - loss: 1.1577 - acc: 0.6365 - val_loss: 0.5236 - val_acc: 0.8667
Epoch 2/5
48000/48000 [==============================] - 0s 8us/step - loss: 0.6463 - acc: 0.8110 - val_loss: 0.4030 - val_acc: 0.8879
Epoch 3/5
48000/48000 [==============================] - 0s 8us/step - loss: 0.5536 - acc: 0.8375 - val_loss: 0.3553 - val_acc: 0.8998
Epoch 4/5
48000/48000 [==============================] - 0s 8us/step - loss: 0.5090 - acc: 0.8527 - val_loss: 0.3474 - val_acc: 0.8978
Epoch 5/5
12000/12000 [==============================] - 1s 43us/step

Generating offsprings
Applying Mutation
Launch new generation?: False
Experiment 5 finished
Global best list



String Model
[[<Layers.FullyConnected: 1>, 216, 1, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 216, 1, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 10, 3, 0, 0, 0, 0, 0]]
<Individual(label = '3' fitness = '2.

60000/60000 [==============================] - 0s 8us/step - loss: 0.0366 - acc: 0.9885 - val_loss: 0.4469 - val_acc: 0.8739
Epoch 30/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0361 - acc: 0.9884 - val_loss: 0.4053 - val_acc: 0.8886
Epoch 31/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0348 - acc: 0.9886 - val_loss: 0.3827 - val_acc: 0.8930
Epoch 32/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0321 - acc: 0.9895 - val_loss: 0.4192 - val_acc: 0.8845
Epoch 33/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0315 - acc: 0.9897 - val_loss: 0.4105 - val_acc: 0.8891
Epoch 34/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0319 - acc: 0.9898 - val_loss: 0.4755 - val_acc: 0.8725
Epoch 35/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0294 - acc: 0.9907 - val_loss: 0.3904 - val_acc: 0.8962
Epoch 36/200
60000/60000 [=====================

60000/60000 [==============================] - 0s 8us/step - loss: 0.0169 - acc: 0.9952 - val_loss: 0.4706 - val_acc: 0.9063
Epoch 89/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0172 - acc: 0.9950 - val_loss: 0.5303 - val_acc: 0.8961
Epoch 90/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0221 - acc: 0.9929 - val_loss: 0.4628 - val_acc: 0.9086
Epoch 91/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0198 - acc: 0.9936 - val_loss: 0.4947 - val_acc: 0.9064
Epoch 92/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0216 - acc: 0.9926 - val_loss: 0.4377 - val_acc: 0.9136
Epoch 93/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0242 - acc: 0.9921 - val_loss: 0.4809 - val_acc: 0.9049
Epoch 94/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0193 - acc: 0.9937 - val_loss: 0.4324 - val_acc: 0.9123
Epoch 95/200
60000/60000 [=====================

60000/60000 [==============================] - 0s 8us/step - loss: 0.0893 - acc: 0.9815 - val_loss: 0.4899 - val_acc: 0.9107
Epoch 148/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0304 - acc: 0.9905 - val_loss: 0.4368 - val_acc: 0.9193
Epoch 149/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0084 - acc: 0.9979 - val_loss: 0.4481 - val_acc: 0.9163
Epoch 150/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0039 - acc: 0.9997 - val_loss: 0.4449 - val_acc: 0.9174
Epoch 151/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0031 - acc: 0.9999 - val_loss: 0.4400 - val_acc: 0.9187
Epoch 152/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0026 - acc: 0.9999 - val_loss: 0.4443 - val_acc: 0.9184
Epoch 153/200
60000/60000 [==============================] - 0s 8us/step - loss: 0.0025 - acc: 0.9999 - val_loss: 0.4441 - val_acc: 0.9187
Epoch 154/200
60000/60000 [==============

### Test on CMAPSS

In [5]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()
print(input_shape)

#mnist datahandler
#dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

save_best_models(global_best_list, global_best_index, 'best_models/cmapss/', train_epochs=200)

(350,)
Launching experiment 1

Generation 1
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 776)               272376    
_________________________________________________________________
dense_462 (Dense)            (None, 576)               447552    
_________________________________________________________________
dense_463 (Dense)            (None, 1)                 577       
Total params: 720,505
Trainable params: 720,505
Non-trainable params: 0
_________________________________________________________________
training with cv
Train on 14868 samples, validate on 20 samples
Epoch 1/5
14868/14868 [==============================] - 9s 599us/step - loss: 4704.7128 - mean_squared_error: 4704.7128 - val_loss: 2550.2302 - val_mean_squared_error: 2550.2302
Epoch 2/5
14868/14868 [==============================] - 0s 12us/step - l

Train on 14401 samples, validate on 20 samples
Epoch 1/5
14401/14401 [==============================] - 9s 640us/step - loss: 3292.8997 - mean_squared_error: 3292.8997 - val_loss: 3220.2236 - val_mean_squared_error: 3220.2236
Epoch 2/5
14401/14401 [==============================] - 0s 17us/step - loss: 530.0646 - mean_squared_error: 530.0646 - val_loss: 2868.5386 - val_mean_squared_error: 2868.5386
Epoch 3/5
14401/14401 [==============================] - 0s 16us/step - loss: 444.3859 - mean_squared_error: 444.3859 - val_loss: 3380.2024 - val_mean_squared_error: 3380.2024
Epoch 4/5
14401/14401 [==============================] - 0s 17us/step - loss: 407.1305 - mean_squared_error: 407.1305 - val_loss: 4084.8608 - val_mean_squared_error: 4084.8608
Epoch 5/5
20/20 [==============================] - 0s 181us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 152)         

14786/14786 [==============================] - 0s 13us/step - loss: 284.5972 - mean_squared_error: 284.5972 - val_loss: 339.9426 - val_mean_squared_error: 339.9426
Epoch 5/5
20/20 [==============================] - 0s 211us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 576)               202176    
_________________________________________________________________
dense_489 (Dense)            (None, 112)               64624     
_________________________________________________________________
dropout_170 (Dropout)        (None, 112)               0         
_________________________________________________________________
dense_490 (Dense)            (None, 936)               105768    
_________________________________________________________________
dense_491 (Dense)            (None, 1000)              937000    
_____________________________________________

14654/14654 [==============================] - 10s 655us/step - loss: 5511.3471 - mean_squared_error: 5511.3471 - val_loss: 5208.4692 - val_mean_squared_error: 5208.4692
Epoch 2/5
14654/14654 [==============================] - 0s 15us/step - loss: 3611.1664 - mean_squared_error: 3611.1664 - val_loss: 4189.8511 - val_mean_squared_error: 4189.8511
Epoch 3/5
14654/14654 [==============================] - 0s 15us/step - loss: 2707.3765 - mean_squared_error: 2707.3765 - val_loss: 3677.3020 - val_mean_squared_error: 3677.3020
Epoch 4/5
14654/14654 [==============================] - 0s 15us/step - loss: 2236.5768 - mean_squared_error: 2236.5768 - val_loss: 3453.8589 - val_mean_squared_error: 3453.8589
Epoch 5/5
20/20 [==============================] - 0s 214us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 256)               89856     
_________________________________

14351/14351 [==============================] - 0s 13us/step - loss: 4332.5789 - mean_squared_error: 4332.5789 - val_loss: 4713.3501 - val_mean_squared_error: 4713.3501
Epoch 3/5
14351/14351 [==============================] - 0s 13us/step - loss: 2051.0969 - mean_squared_error: 2051.0969 - val_loss: 2657.6831 - val_mean_squared_error: 2657.6831
Epoch 4/5
14351/14351 [==============================] - 0s 13us/step - loss: 1254.8696 - mean_squared_error: 1254.8696 - val_loss: 2198.7781 - val_mean_squared_error: 2198.7781
Epoch 5/5
20/20 [==============================] - 0s 223us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 256)               89856     
_________________________________________________________________
dense_512 (Dense)            (None, 1)                 257       
Total params: 90,113
Trainable params: 90,113
Non-trainable params: 0
___________

...

training with cv
Train on 14588 samples, validate on 20 samples
Epoch 1/5
14588/14588 [==============================] - 17s 1ms/step - loss: 6268.7505 - mean_squared_error: 6268.7505 - val_loss: 7653.2876 - val_mean_squared_error: 7653.2876
Epoch 2/5
14588/14588 [==============================] - 0s 19us/step - loss: 1843.4518 - mean_squared_error: 1843.4518 - val_loss: 3466.7153 - val_mean_squared_error: 3466.7153
Epoch 3/5
14588/14588 [==============================] - 0s 19us/step - loss: 852.5070 - mean_squared_error: 852.5070 - val_loss: 2666.5437 - val_mean_squared_error: 2666.5437
Epoch 4/5
14588/14588 [==============================] - 0s 19us/step - loss: 529.9385 - mean_squared_error: 529.9385 - val_loss: 2333.9001 - val_mean_squared_error: 2333.9001
Epoch 5/5
20/20 [==============================] - 0s 411us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (N

Printing shapes

Training data (X, y)
(18231, 350)
(18231, 1)
Testing data (X, y)
(100, 350)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.59060403 -0.0455243  -0.27982732 ... -0.27272727  0.25581395
   0.50126369]
 [-0.36912752  0.0629156  -0.18014129 ... -0.09090909  0.11627907
   0.46222971]
 [-0.23489933 -0.13299233 -0.13854003 ... -0.27272727  0.31782946
   0.55293457]
 [-0.23489933 -0.39897698 -0.2299843  ... -0.81818182  0.34883721
   0.09491716]
 [-0.22147651 -0.39590793 -0.05926217 ... -0.27272727  0.2248062
   0.30721707]]
[[130.]
 [130.]
 [130.]
 [130.]
 [130.]]
Testing data (X, y)
[[-0.39597315 -0.37084399 -0.11342229 ... -0.27272727  0.03875969
   0.29458017]
 [ 0.42281879 -0.02148338 -0.1189168  ... -0.63636364  0.03875969
   0.0322943 ]
 [ 0.46308725 -0.05524297  0.08477237 ...  0.09090909  0.2248062
   0.06655434]
 [-0.14765101  0.13452685  0.05965463 ...  0.09090909 -0.31782946
   0.02190396]
 [ 0.22147651  0.06240409  0.32849294 ... -0.09090909 -0.0542

18231/18231 [==============================] - 0s 19us/step - loss: 202.3848 - mean_squared_error: 202.3848
Epoch 60/200
18231/18231 [==============================] - 0s 19us/step - loss: 206.3923 - mean_squared_error: 206.3923
Epoch 61/200
18231/18231 [==============================] - 0s 19us/step - loss: 197.0833 - mean_squared_error: 197.0833
Epoch 62/200
18231/18231 [==============================] - 0s 19us/step - loss: 202.0678 - mean_squared_error: 202.0678
Epoch 63/200
18231/18231 [==============================] - 0s 19us/step - loss: 197.5218 - mean_squared_error: 197.5218
Epoch 64/200
18231/18231 [==============================] - 0s 19us/step - loss: 194.5975 - mean_squared_error: 194.5975
Epoch 65/200
18231/18231 [==============================] - 0s 19us/step - loss: 188.6376 - mean_squared_error: 188.6376
Epoch 66/200
18231/18231 [==============================] - 0s 19us/step - loss: 196.6366 - mean_squared_error: 196.6366
Epoch 67/200
18231/18231 [===================

18231/18231 [==============================] - 0s 19us/step - loss: 153.8015 - mean_squared_error: 153.8015
Epoch 127/200
18231/18231 [==============================] - 0s 19us/step - loss: 149.5279 - mean_squared_error: 149.5279
Epoch 128/200
18231/18231 [==============================] - 0s 19us/step - loss: 147.8248 - mean_squared_error: 147.8248
Epoch 129/200
18231/18231 [==============================] - 0s 19us/step - loss: 142.1704 - mean_squared_error: 142.1704
Epoch 130/200
18231/18231 [==============================] - 0s 19us/step - loss: 145.5233 - mean_squared_error: 145.5233
Epoch 131/200
18231/18231 [==============================] - 0s 19us/step - loss: 143.3670 - mean_squared_error: 143.3670
Epoch 132/200
18231/18231 [==============================] - 0s 19us/step - loss: 135.9446 - mean_squared_error: 135.9446
Epoch 133/200
18231/18231 [==============================] - 0s 19us/step - loss: 144.9571 - mean_squared_error: 144.9571
Epoch 134/200
18231/18231 [===========

18231/18231 [==============================] - 0s 19us/step - loss: 108.5156 - mean_squared_error: 108.5156
Epoch 194/200
18231/18231 [==============================] - 0s 19us/step - loss: 111.2878 - mean_squared_error: 111.2878
Epoch 195/200
18231/18231 [==============================] - 0s 19us/step - loss: 114.3059 - mean_squared_error: 114.3059
Epoch 196/200
18231/18231 [==============================] - 0s 19us/step - loss: 106.4969 - mean_squared_error: 106.4969
Epoch 197/200
18231/18231 [==============================] - 0s 19us/step - loss: 118.5806 - mean_squared_error: 118.5806
Epoch 198/200
18231/18231 [==============================] - 0s 19us/step - loss: 115.5186 - mean_squared_error: 115.5186
Epoch 199/200
18231/18231 [==============================] - 0s 19us/step - loss: 107.8234 - mean_squared_error: 107.8234
Epoch 200/200
100/100 [==============================] - 0s 322us/step
Saved models for dataset 1 to disk


In [6]:
#save_best_models(global_best_list, global_best_index, train_epochs=200)